Welcome to module 2.2! In the last module, we investigated how to use `scikit-learn` to build a simple Naive Bayes model to perform sentiment analysis. In this notebook we will look behind the scenes: we will build our own Naive Bayes model from scratch. We will reuse some of the code we wrote in the previous notebook - but this time, we'll write the maths from scratch.

## ❓ Pre-module quiz

Why is Naive Bayes "naive"?

A. Because it's the most basic, i.e. "naive" classifier we can build

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are independent of each other

C. Because the guy who invented it tought it was a cool name

D. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are dependent of each other

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>The correct answer is B - Naive Bayes assumes that the probability of finding a certain word is independent from the probability of finding another word. So, for example, in the domain of movie reviews, it assumes that the probabilities of finding the words <code>Indiana</code> and <code>Jones</code> are not correlated, even if in practice we know that this is not the case.</p>

</details> 


# Python for Computational Linguists 1.2: Breaking down Naive Bayes

## Introduction

### Recap of last notebook

Module 2.1 introduced lots of new concepts and libraries - pre-processing, feature vectors, `numpy`, and so on. In fact, we have seen how to import data from a research dataset, how to clean them by removing punctuation and stop-words, how to use `numpy` to prepare test and training data for a model, and how to use `scikit-learn` to train, test and evaluate a simple Naive Bayes classifier for sentiment analysis.

However, we didn't look 'behind the scenes' at how Naive Bayes actually works. This module will guide you through your first hand-written machine learning model, by showing you how to write the maths for Naive Bayes yourself. We will introduce the concept of "class", and we'll debug and improve our model by adding smoothing.

If you're interested, you can also just go and look the code of `scikit-learn`'s implementation [here](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/naive_bayes.py#L669)!

### Naive Bayes refresher

> Note: this section borrows heavily from the Naive Bayes chapter of the lecture notes. Please refer to them or to the J&M for more details.

Naive Bayes is a simple classifier based on two assumptions:
- The **bag-of-words** assumption: word ordering doesn't matter. We represent each document in our dataset as a list of pairs $(word_i,frequency_i)$.
- The **conditional independence** assumption: the probability of one word appearing in a sentence is by no means correlated to the occurrence of another word.

These two assumptions heavily simplify our model, since they completely disregard grammar and and eventual domain knowledge (e.g. the `Indiana Jones` example in the pre-module quiz); but as we know now, they allow us to build a surprisingly efficient classifier.

Let's quickly go through the maths of Naive Bayes. Remember that given a document $d$ and a set of classes $C$, the probability the document belonging to a class $c \in C$ is $\hat{c}$, and it is defined as:

$$ \hat{c} = \text{argmax}_{c \in C} P(c \mid d)$$

However, the Bayes Rule tells us that

$$ P(c \mid d) = \frac{P(c) \ P(d \mid c)}{P(d)} $$

Allowing us to rewrite 

$$\begin{align} 
\hat{c} &= \text{argmax}_{c \in C} P(c \mid d) \\
        &= \text{argmax}_{c \in C} \frac{P(c) \ P(d \mid c)}{P(d)}
\end{align}$$

However, the probability $d$ is constant for each class $c$, hence we can remove it, leaving only:

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c) \ P(d \mid c)}_\text{likelihood}
    \underbrace{P(d)}_\text{prior}
$$

Where the $prior$ is the **prior probability** of the class $c$ and the $likelihood$ is the probability of finding $d$ given the class $c$.

Using words as features, we can represent $d$ as a list of words $w_1, \dots , w_n$, hence 

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c) \ P(w_1, \dots , w_n \mid c)}_\text{likelihood} 
    \underbrace{P(d)}_\text{prior}
$$

However, $P(w_1, \dots , w_n \mid c)$ may be prohibitively hard to calculate, since we would need to estimate the probability of every possible combination of words. Here, the **conditional independence** assumption comes to the rescue, assuming the probability of the words (i.e. features) are independent, allowing us to finally rewrite

$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{P(c) \ P(w_1 \mid c) \times \dots \times P(w_n \mid c)}_\text{likelihood} 
       \\
       &= \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

What does all of this mean in practice? Well, that if we have a document $d$, all we need to know to classify it is:
- The *priors*, i.e. the probability of document $d$ to belong to each class $c$
- The *likelihoods*, i.e. the probabilities for each word $w_i$ of the document to belong to each class $c$.

## Naive Bayes: a simple implementation

Let's begin with a simple example from the Post Lecture exercises (taken from J&M-3, exercise 4.2). Given the following short movie reviews, each labeled with a genre, either comedy or action:

| review                      | class  |
|-----------------------------|--------|
| fun, couple, love, love     | comedy |
| fast, furious, shoot        | action |
| couple, fly, fast, fun, fun | comedy |
| furious, shoot, shoot, fun  | action |
| fly, fast, shoot, love      | action |

And a new document D: 

| review                     | class  |
|----------------------------|--------|
| fast, couple, shoot, fly   | ?      |

Compute the most likely class for D.

Let's start by saving our documents in some vectors:

In [73]:
train_docs = [
    ['fun', 'couple', 'love', 'love'],
    ['fast', 'furious', 'shoot'],
    ['couple', 'fly', 'fast', 'fun', 'fun'],
    ['furious', 'shoot', 'shoot', 'fun'],    
    ['fly', 'fast', 'shoot', 'love']]

train_labels = ['comedy', 'action', 'comedy', 'action', 'action']

### Computing the priors

Remember what we needed to do? The first step is to compute the **priors**. Let's do that with a simple function:

In [77]:
# What are our classes?
classes = set(train_labels)
print(classes)

# initialise the priors
priors = {}
for _class in classes:
    priors[_class] = 0

# count how many train example in each class
for _class in classes:
    for label in train_labels:
        if _class == label:
            priors[_class] += 1

print(priors)

{'action', 'comedy'}
{'action': 3, 'comedy': 2}


> **<h3>💻 Try it yourself!</h3>**

Now the priors are not *normalised*, i.e. we have to bring each prior in the range $[0,1]$. Can you do that in the following cell?

In [ ]:
# write your code here

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
for _class in classes:
    priors[_class] = priors[_class] / len(train_labels)
    </code></pre></p>

</details> 

If you've got the priors correct, you should have $P(comedy) = 0.4$ and $P(action)=0.6$.

In [ ]:
print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now let's wrap everything nicely into a function. Can you complete the cell below?

In [ ]:
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''

    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}

    # ...?

    # count how many train example in each class
    
    # ...?
    
    # normalise the priors
    
    # ...?
    
    return priors

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''
    
    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}
    for _class in classes:
        priors[_class] = 0

    # count how many train example in each class
    for _class in classes:
        for label in labels:
            if _class == label:
                priors[_class] += 1
    
    # normalise the priors
    for _class in classes:
        priors[_class] = priors[_class] / len(labels)
        
    return priors
    </code></pre></p>

</details> 

In [78]:
priors = compute_priors(train_labels)
print(priors)

{'action': 0.6, 'comedy': 0.4}


### Find the vocabulary

Since we need to compute the likelihoods for all words in our vocabulary, we need to find all the words in our corpus first. Let's build our vocabulary with a simple function:

In [79]:
def create_vocabulary(lines):
    '''
    Creates a vocabulary from test.
    
    Parameters
    ----------
    lines: a list of lists of words
    
    Returns
    -------
    a set with all the words in the lines.
    
    '''
    
    words = set()
    for line in lines:
        for word in line:
            words.add(word)
            
    return words

In [80]:
create_vocabulary(train_docs)

{'couple', 'fast', 'fly', 'fun', 'furious', 'love', 'shoot'}

### Computing the likelihoods

Now we need to compute the likelihoods of the words w.r.t. to each class. To do that, we can build a `dict` for each class, where `dict[word] = P(word|class)`. To do that we can slightly modify the function `create_vocab_dict` that we defined in [Module 1.4](../../module_1/module_1.4/module_1.4.ipynb). The function we defined was the following:

In [8]:
def create_vocab_dict(lines):
    '''
    Collect vocabulary counts from text

    Parameters
    ----------
    f_processed_arg: a list of lists of words

    Returns
    -------
    a dictionary with words (str) as keys and counts(int) as values
    vocab={
    'SONNETS': 1
    }
    '''
    vocab={}# create an empty vocabulary dictionary to store words as keys and counts as values later. 
    for line in lines:
        for word in line:
            if word in vocab:
                vocab[word]+=1 # update the count for an existing word
            else:
                vocab[word]=1 # initilize the count for a new word
    return vocab

In [9]:
print(create_vocab_dict(train_docs))

{'fun': 4, 'couple': 2, 'love': 3, 'fast': 3, 'furious': 1, 'shoot': 4, 'fly': 2, 'curious': 1}


> **<h3>💻 Try it yourself!</h3>**

How can we modify this function to give us the likelihoods for each class? Modify the function below, directly derived from `create_vocab_dict`, to return the likelihoods instead of the raw counts.

In [18]:
def compute_likelihoods(lines, vocabulary):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    f_processed_arg: a list of list of words

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # create an empty vocabulary dictionary to store words 
    # as keys and counts as values later. 
    
    likelihoods = {}
    
    # populate the likelihoods
    # hint: iterate through the vocabulary and initialise
    # a new element of the likelihoods dict to 0

    
    
    # ~      end exercise    ~ #    
    
    # Now we iterate through the lines
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    
    for word in likelihoods:
        likelihoods[word] = # Write your code here
    
    # ~      end exercise    ~ #

    return likelihoods

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines, vocabulary):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    f_processed_arg: a list of list of words

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # create an empty vocabulary dictionary to store words 
    # as keys and counts as values later. 
    
    likelihoods = {}
    
    # populate the likelihoods
    # hint: iterate through the vocabulary and initialise
    # a new element of the likelihoods dict to 0
    for word in vocabulary:
        likelihoods[word] = 0
    
    
    # ~      end exercise    ~ #    
    
    # Now we iterate through the lines
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    for line in lines:
        for word in line:
            total_tokens +=1
    
    for word in likelihoods:
        likelihoods[word] = likelihoods[word]/total_tokens
    
    # ~      end exercise    ~ #

    return likelihoods
    </code></pre></p>

</details> 

In [23]:
print(compute_likelihoods(train_docs, create_vocabulary(train_docs)))

{'curious': 0.05, 'fly': 0.1, 'shoot': 0.2, 'love': 0.15, 'fun': 0.2, 'fast': 0.15, 'furious': 0.05, 'couple': 0.1}


Note that this method computes the likelihoods of the words in the whole vocabulary, irregardless of the classes. To get the likelihoods for each class we need to do as follows:

In [82]:
target_class = 'action'
target_docs = []
vocabulary = create_vocabulary(train_docs)

# enumerate builds an (index, doc) list, hence allowing
# us to retrieve the label for each doc
for i, doc in enumerate(train_docs):
    if train_labels[i] == target_class:
        target_docs.append(doc)
        
print(target_docs)
print(compute_likelihoods(target_docs, vocabulary))

[['fast', 'furious', 'shoot'], ['furious', 'shoot', 'shoot', 'fun'], ['fly', 'fast', 'shoot', 'love']]


TypeError: compute_likelihoods() missing 1 required positional argument: 'smoothing'

### The training function

> **<h3>💻 Try it yourself!</h3>**

Now we have all the instruments to build our training function. Can you complete it?

In [28]:
def train_naive_bayes(documents, labels):
    
    classes = set(labels)
    
    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)
    
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        # put your code here
        for #...
        
        # compute the likelihood of this class
        likelihoods[_class] = # hint: use compute_likekihood defined above
        
    # ~      end exercise    ~ #
        
    return priors, likelihoods    

SyntaxError: invalid syntax (<ipython-input-28-531bed73424a>, line 21)

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def train_naive_bayes(documents, labels):

    classes = set(labels)

    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)

    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}

    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)

        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs, vocabulary)

    return priors, likelihoods 
    </code></pre></p>

</details> 

In [30]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels)

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

Priors:
{'action': 0.6, 'comedy': 0.4}

Likelihoods:
{'action': {'curious': 0.09090909090909091, 'fly': 0.09090909090909091, 'shoot': 0.36363636363636365, 'love': 0.09090909090909091, 'fun': 0.09090909090909091, 'fast': 0.18181818181818182, 'furious': 0.09090909090909091, 'couple': 0.0}, 'comedy': {'curious': 0.0, 'fly': 0.1111111111111111, 'shoot': 0.0, 'love': 0.2222222222222222, 'fun': 0.3333333333333333, 'fast': 0.1111111111111111, 'furious': 0.0, 'couple': 0.2222222222222222}}


### Predict unknown classes

So now we have trained our model. How can we predict the likelihood of new sentences belonging to each class? Remember from above that

$$
\hat{c} = \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
$$

So, to predict the class of our new sentence `fast, couple, shoot, fly`, we need to:
- for each class `c`, we need to calculate `prob_c` by
    - multiplying the probability of each word `w` given that class `c`

Then, select the maximum of our `prob_c`s. So, can we predict the class of `fast, couple, shoot, fly`? Let's write a function to do this.

In [38]:
def bayes_predict(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    
    Return
    ------
    The probability for each class.
    '''
    
    classes_probabilities = {}

    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():
        
        # initialise the probability of a class to its prior
        prob_class = prior
        for word in document:
            if word in likelihoods[label]:
                # multiply the prior for the likelihood of each word
                prob_class = prob_class*likelihoods[label][word]
        classes_probabilities[label] = prob_class
        
    return classes_probabilities
    

Good! Let's try to a new document with some random words:

In [44]:
document = ['fast', 'fun', 'love', 'fly']
bayes_predict(document, priors, likelihoods)

{'action': 8.19616146438085e-05, 'comedy': 0.0003657978966620942}

So for this document, `comedy` is the most likely class! 

Now let's try with the document for the assignment:

In [45]:
document = ['fast', 'couple', 'shoot', 'fly']
bayes_predict(document, priors, likelihoods)

{'action': 0.0, 'comedy': 0.0}

Now the classes are both zero! How come? Well, we didn't apply any smoothing, so obviously at some point we are multiplying the likelihoods by zero, since $P(couple \ \mid action) = 0$ and $P(shoot \mid love) = 0$.

### Adding 1-smoothing

> **<h3>💻 Try it yourself!</h3>**

How can we modify `compute_likelihoods` to add 1-smoothing? Please update the function below.

Note that:
- We added a parameter (`smoothing`) to select the smoothing mode
- You will need to change how to normalise the likelihoods.

In [92]:
def compute_likelihoods(lines, vocabulary, smoothing):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    smooething: the smoothing method to use

    Returns
    -------
    a dictionary with the probability for each class
    '''
    
    likelihoods = {}

    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
 

    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1

    # Apply smoothing, if needed
    for word in likelihoods:
        if smoothing == 'none':
            
            for line in lines:
                for word in line:
                    total_tokens +=1

            for word in likelihoods:
                likelihoods[word] = likelihoods[word]/total_tokens
            
        elif smoothing == 'add1':
            # -------------------------#
            #      E X E R C I S E     #
            # -------------------------#
            # calculate the smoothing parameter for each word.
            smoothing_param = # ...?
            
            likelihoods[word] = # ...?
            # ~      end exercise    ~ #
        else:
            print('Unknown smoothing!')
            return

    return likelihoods

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines, vocabulary, smoothing):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    smooething: the smoothing method to use

    Returns
    -------
    a dictionary with the probability for each class
    '''
    
    likelihoods = {}

    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
 

    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1

    # Apply smoothing, if needed
    for word in likelihoods:
        if smoothing == 'none':
            
            for line in lines:
                for word in line:
                    total_tokens +=1

            for word in likelihoods:
                likelihoods[word] = likelihoods[word]/total_tokens
            
        elif smoothing == 'add1':
            # -------------------------#
            #      E X E R C I S E     #
            # -------------------------#
            # calculate the smoothing parameter for each word.
            smoothing_param = total_tokens + len(vocabulary)
            
            likelihoods[word] = (likelihoods[word] + 1)/smoothing_param
            # ~      end exercise    ~ #
        else:
            print('Unknown smoothing!')
            return

    return likelihoods
    </code></pre></p>

</details> 

Now let's update `train_naive_bayes` to instruct it to use smoothing, and let's see if the results we obtain are correct:

In [89]:
def train_naive_bayes(documents, labels, smoothing):

    classes = set(labels)

    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)

    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}

    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)

        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs, vocabulary, smoothing)

    return priors, likelihoods 

In [90]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels, 'add1')

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

Priors:
{'action': 0.6, 'comedy': 0.4}

Likelihoods:
{'action': {'fly': 0.1111111111111111, 'shoot': 0.2777777777777778, 'love': 0.1111111111111111, 'fun': 0.1111111111111111, 'fast': 0.16666666666666666, 'furious': 0.16666666666666666, 'couple': 0.05555555555555555}, 'comedy': {'fly': 0.125, 'shoot': 0.0625, 'love': 0.1875, 'fun': 0.25, 'fast': 0.125, 'furious': 0.0625, 'couple': 0.1875}}


In [91]:
document = ['fast', 'couple', 'shoot', 'fly']
bayes_predict(document, priors, likelihoods)

{'action': 0.00017146776406035664, 'comedy': 7.324218750000001e-05}

Great! We successfully implemented Naive Bayes in Python. 

### Improving Naive Bayes: `argmax` and log-likelihoods

### Other smoothing functions

See [this paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.8978&rep=rep1&type=pdf)

## Re-classify the *Thumbs up?* dataset

### Recap of loader functions

### Applying our model

### Differences between our model and `scikit-learn`

## Wrapping up